# Agents

Agents wrap a model and give it access to a set of tools. These tools may access additional data sources, APIs, or functionality. The model is used to determine which of the tools to use to complete a task.

The agent you will create will be able to chat about movies and search YouTube for movie trailers.

#### Tools

A tool is a specific abstraction around a function that makes it easy for a language model to interact with it. Langchain provides several tools out of the box, and you can create tools to extend the functionality of your agents.

You will use the YouTube Tool to search YouTube for movie trailers.

In [19]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.schema import StrOutputParser
from langchain_community.tools import YouTubeSearchTool
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_community.graphs import Neo4jGraph
from uuid import uuid4
from langsmith import Client
import os

In [11]:
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")

Session ID: 1529e8eb-1b1c-458e-93f9-232e20a9e2b0


In [12]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

In [13]:
graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="12345678"
)

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/neo4j/_sync/driver.py:511: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  deprecation_warn(


In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie expert. You find movies from a genre or plot.",
        ),
        ("human", "{input}"),
    ]
)

movie_chat = prompt | llm | StrOutputParser()

In [15]:
youtube = YouTubeSearchTool()

def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

def call_trailer_search(input):
    input = input.replace(",", " ")
    return youtube.run(input)

#### Creating tools
Tools are interfaces that an agent can interact with. You can create custom tools able to perform any functionality you want.

In this example, the Tool is created from a function. The function is the movie_chat.invoke method.

In [16]:
tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=movie_chat.invoke,
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word trailer. Return a link to a YouTube video.",
        func=call_trailer_search,
    ),
]

The name and description help the LLM select which tool to use when presented with a question. The func parameter is the function that will be called when the tool is selected. The return_direct flag indicates that the tool will return the result directly.

Agents support multiple tools, so you pass them to the agent as a list (tools).

Initializing an agent
The following code creates the agent:

In [17]:
client = Client()

agent_prompt = client.pull_prompt("hwchase17/react-chat")
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

chat_agent = RunnableWithMessageHistory(
    agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

There are different types of agents that you can create. This example creates a ReAct - Reasoning and Acting) agent type.

An agent requires a prompt. You could create a prompt, but in this example, the program pulls a pre-existing prompt from the Langsmith Hub.

The `hwcase17/react-chat` prompt instructs the model to provide an answer using the tools available in a specific format.

The `create_react_agent` function creates the agent and expects the following parameters:

- The `llm` that will manage the interactions and decide which tool to use
- The `tools` that the agent can use
- The `prompt` that the agent will use

The `AgentExecutor` class runs the agent. It expects the following parameters:

- The `agent` to run
- The `tools` that the agent can use
- The `memory` which will store the conversation history

In [ ]:
while True:
    q = input("> ")

    response = chat_agent.invoke(
        {
            "input": q
        },
        {"configurable": {"session_id": SESSION_ID}},
    )
    
    print(response["output"])